## Process Data


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/smvinodkumar910/market-mirror/blob/main/backend/03_processing_data_01.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fsmvinodkumar910%2Fmarket-mirror%2Frefs%2Fheads%2Fmain%2Fbackend%2F03_processing_data_01.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/smvinodkumar910/market-mirror/refs/heads/main/backend/03_processing_data_01.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/smvinodkumar910/market-mirror/blob/main/backend/03_processing_data_01.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/475654/github-color.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    # Support for third party widgets
    from google.colab import auth, output

    auth.authenticate_user()
    output.enable_custom_widget_manager()

### Setting-up Environment

* Please change the variables `PROJECT_ID`, `BUCKET_NAME`, `LOCATION` details to your own project as required.

In [1]:
import os

PROJECT_ID = "market-mirror-dev"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
BUCKET_NAME = "marke-mirror-dev-data"  # @param {type: "string", placeholder: "[your-bucket-name]", isTemplate: true}
LOCATION = "US"  # @param {type: "string", placeholder: "[your-region]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

if not LOCATION or LOCATION == "[your-region]":
    LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "US")


In [2]:
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_REGION'] = LOCATION

In [3]:
BQ_BRONZE_DATASET = "APP_MARKET_BRONZE" # @param {type: "string", placeholder: "[bronze-dataset]", isTemplate: true}
BQ_SILVER_DATASET = "APP_MARKET_SILVER" # @param {type: "string", placeholder: "[silver-dataset]", isTemplate: true}
BQ_GOLD_DATASET = "APP_MARKET_GOLD" # @param {type: "string", placeholder: "[gold-dataset]", isTemplate: true}

In [ ]:
import bigframes.pandas as bpd
import bigframes.bigquery as bbq
from bigframes.ml import llm

bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = LOCATION

In [ ]:
# Read reviews table from SILVER layer
review_df = bpd.read_gbq(f'{PROJECT_ID}.{BQ_SILVER_DATASET}.T_APP_REVIEWS')

In [6]:
review_df.info()

<class 'bigframes.dataframe.DataFrame'>
Index: 69183 entries, 0 to 69182
Data columns (total 6 columns):
  #  Column       Non-Null Count    Dtype
---  -----------  ----------------  -------
  0  id           4888 non-null     Int64
  1  app_name     69183 non-null    string
  2  app_genre    3018 non-null     string
  3  review_text  42315 non-null    string
  4  rating       4888 non-null     Int64
  5  sentiment    37432 non-null    string
dtypes: Int64(2), string(4)
memory usage: 3874248 bytes


In [7]:
review_df.isna().sum()

id             64295
app_name           0
app_genre      66165
review_text    26868
rating         64295
sentiment      31751
dtype: Int64

* Above details  shows that `app_name` column only not null. All other column have null. We need to fix these by,

1. `id` - we can generate owr own unique id.
2. `app_genre`- we can get `app_genre` from the Goole Product details table we have in SILVER layer
    We can use Bigquery GenAI functions to predict `app_genre` based on app description or app name.
3. `review_text` - The whole point of having this table is to understand users sentiment. records having Null review_text is useless for us. We can remove these records.
4. ratings - 
5. sentiment - we can utilize Bigquery GenAI functions to generate Sentiment of the review.

In [ ]:
# Removing the records where review_text is null.
review_df_removed_null_review = review_df[~review_df['review_text'].isna()]

In [9]:
review_df_removed_null_review.isna().sum()

id             37427
app_name           0
app_genre      39297
review_text        0
rating         37427
sentiment       4888
dtype: Int64

* Fixing the `app_genre` column

In [ ]:
#Reading Google app details table from SILVER layer
google_apps_df = bpd.read_gbq(f'{PROJECT_ID}.{BQ_SILVER_DATASET}.T_GOOGLE_APP_DETAILS',
                              columns=['title','genre']).drop_duplicates()

In [ ]:
#left join the review table and product details table 
review_df_removed_null_review['join']= review_df_removed_null_review.app_name.str.strip().str.lower()
google_apps_df['join']= google_apps_df.title.str.strip().str.lower()
merged_review_df = bpd.merge(review_df_removed_null_review, google_apps_df, on='join', how='left')

In [ ]:
# Update the `app_genre` field with the `genre` column from the Goole app details dataset
merged_review_df['app_genre'] = merged_review_df['app_genre'].fillna(merged_review_df['genre'])

In [13]:
merged_review_df.isna().sum()

id             37581
app_name           0
app_genre      38535
review_text        0
rating         37581
sentiment       4891
join               0
title          41406
genre          41406
dtype: Int64

* Even after updating based on App_name, we can see that still 38k+ records having null `app_genre`
* We can utilize the ML Functions in Bigquery to enrich the dataset.

In [ ]:
# collecting the unique list of genres available from google app details dataseet
genre_list = google_apps_df.genre.drop_duplicates().to_list()

In [15]:
genre_list.extend(merged_review_df[~merged_review_df.app_genre.isna()].app_genre.drop_duplicates().to_list())

In [16]:
genre_list = list(set(genre_list))

In [ ]:
# We have 49 unique genere available
len(genre_list)

49

In [ ]:
# converting the list of Genre into a String joined by ','
genre_list_concat = ','.join(genre_list)

In [ ]:
# Getting the unique list of APPs without genre informationn in the review table
unique_apps_without_genre = merged_review_df[merged_review_df.app_genre.isna()].app_name.drop_duplicates().to_frame(name='app_name')

In [ ]:
# Creating a prompt column in the dataframe for GenAI processing
unique_apps_without_genre['prompt'] ="""predict which genre the given app belongs to.Pick the most possible genre.
app_name : """+ unique_apps_without_genre.app_name + """
Reply with just the genre predicted for the given app name. genre can be any one from the following list - genre_list:""" + genre_list_concat

In [ ]:
# view some sample data
unique_apps_without_genre[0:5]

,app_name,prompt
0,BEST CAR SOUNDS,predict which genre the given app belongs to. ...
1,BeyondMenu Food Delivery,predict which genre the given app belongs to. ...
2,591房屋交易-租屋、中古屋、新建案、實價登錄、別墅透天、公寓套房、捷運、買房賣房行情、房價...,predict which genre the given app belongs to. ...
4,Calorie Counter & Diet Tracker,predict which genre the given app belongs to. ...
5,Foursquare Swarm: Check In,predict which genre the given app belongs to. ...


In [ ]:
# create a model object using llm.GeminiTextGenerator() using the connection we have created already in previous notebook
model = llm.GeminiTextGenerator(model_name='gemini-2.0-flash-001', connection_name='us.vertex-remote-models')

In [ ]:
# Executing the model.predict() method with desired output schema to enrich the data with correct app_genre
llm_response = model.predict(X=unique_apps_without_genre,
              output_schema={'app': 'string', 'app_genre': 'string'})


/usr/local/lib/python3.11/dist-packages/bigframes/core/array_value.py:108: PreviewWarning: JSON column interpretation as a custom PyArrow extention in
`db_dtypes` is a preview feature and subject to change.
  warnings.warn(msg, bfe.PreviewWarning)


In [ ]:
# view the sample response generated
llm_response.head()

,app,app_genre,full_response,status,app_name,prompt
0,BEST CAR SOUNDS,Auto & Vehicles,"{""candidates"":[{""avg_logprobs"":-0.005561906557...",,BEST CAR SOUNDS,predict which genre the given app belongs to.P...
1,BeyondMenu Food Delivery,Food & Drink,"{""candidates"":[{""avg_logprobs"":-0.000159307383...",,BeyondMenu Food Delivery,predict which genre the given app belongs to.P...
2,591房屋交易-租屋、中古屋、新建案、實價登錄、別墅透天、公寓套房、捷運、買房賣房行情、房價...,House & Home,"{""candidates"":[{""avg_logprobs"":-1.609029255412...",,591房屋交易-租屋、中古屋、新建案、實價登錄、別墅透天、公寓套房、捷運、買房賣房行情、房價...,predict which genre the given app belongs to.P...
4,<NA>,Health & Fitness,"{""candidates"":[{""avg_logprobs"":-0.011345310012...",,Calorie Counter & Diet Tracker,predict which genre the given app belongs to.P...
5,Foursquare Swarm: Check In,Social,"{""candidates"":[{""avg_logprobs"":-0.005781240761...",,Foursquare Swarm: Check In,predict which genre the given app belongs to.P...


In [ ]:
# Extract the output information from llm response as a dataframe
llm_response_subset = llm_response[['app_name','app_genre']]
llm_response_subset = llm_response_subset.rename(columns={'app_genre':'app_genre_llm'})

In [ ]:
# join the response provided by llm with the review dataframe
merged_review_df_with_genre = bpd.merge(merged_review_df,llm_response_subset,on='app_name',how='left')


In [ ]:
# Update the `app_genre` field where its null using the app_genre provided by llm
merged_review_df_with_genre['app_genre'] = merged_review_df_with_genre['app_genre'].fillna(merged_review_df_with_genre['app_genre_llm'])

In [50]:
merged_review_df_with_genre.isna().sum()

id               37581
app_name             0
app_genre            0
review_text          0
rating           37581
sentiment         4891
join                 0
title            41406
genre            41406
app_genre_llm     3850
dtype: Int64

In [ ]:
# Finallly we can see `app_genre` field is fixed, without any null records
# Now we can remove unnecessary columns from the dataframe
merged_review_df_with_genre = merged_review_df_with_genre.drop(columns=['rating','join','title','genre','app_genre_llm'])



In [ ]:
#Write the cleaned app_review table to BQ SILVER layer with prefixed _CLEANED 
merged_review_df_with_genre.to_gbq(destination_table=f"{PROJECT_ID}.{BQ_SILVER_DATASET}.T_APP_REVIEWS_CLEANED",if_exists='replace')

'market-mirror-dev.APP_MARKET_SILVER.T_APP_REVIEWS_CLEANED'

In [53]:
merged_review_df_with_genre.isna().sum()

id             37581
app_name           0
app_genre          0
review_text        0
sentiment       4891
dtype: Int64

* With the above results, we can see that, sentiment is null for 4k+ records. We need to utilize BQ Ml functions to get sentiment for those reviews.
* Also, the id column is null, for 37k+ records, we can genreate out id for the same.

In [ ]:
# reset index to apply to the `id` column
merged_review_df_with_genre = merged_review_df_with_genre.reset_index()
merged_review_df_with_genre = merged_review_df_with_genre.drop(columns=['id']).rename(columns={'index':'id'})

In [ ]:
# Write the results to BQ silver layer
merged_review_df_with_genre.to_gbq(destination_table=f"{PROJECT_ID}.{BQ_SILVER_DATASET}.T_APP_REVIEWS_CLEANED", if_exists='replace')

'market-mirror-dev.APP_MARKET_SILVER.T_APP_REVIEWS_CLEANED'

In [47]:
merged_review_df_with_genre.sentiment.value_counts()

sentiment
Positive    24081
Negative     8342
Neutral      5158
Name: count, dtype: Int64

* In the below step we are executing a Bigquery SQL, to invoke the `AI.GENERATE` function to generate Sentiment for each review and storing it in a separarate table called `T_APP_REVIEWS_SENTIMENTS` 

**Please make sure the PROJECT_NAME field here is replaced with your project_id and correct SILVER dataset name is given.**

In [ ]:
%%bigquery
create or replace table `PROJECT_NAME.APP_MARKET_SILVER.T_APP_REVIEWS_SENTIMENTS`
AS
select id, app_name, review_text,
AI.GENERATE(prompt=>concat('check the sentiment of the user review for the app and give me answer in just one word on whether the review is [Positive   Negative or Neutral app_name: ', app_name, '\r\n review_text : ', review_text ),
connection_id => 'us.vertex-remote-models',
endpoint => 'gemini-2.5-flash'
).result
 from `PROJECT_NAME.APP_MARKET_SILVER.T_APP_REVIEWS_CLEANED`
where sentiment is null --and id in (27394, 27294, 41092)
;



* In the below cell we are updating the `sentiment` column in `T_APP_REVIEWS_CLEANED` table with the sentiments provided by the LLM.

**Please make sure the PROJECT_NAME is replaced with your project_id**

In [ ]:
%%bigquery
update `PROJECT_NAME.APP_MARKET_SILVER.T_APP_REVIEWS_CLEANED` tgt
set tgt.sentiment = src.result
from `PROJECT_NAME.APP_MARKET_SILVER.T_APP_REVIEWS_SENTIMENTS` src
where tgt.id = src.id
and tgt.sentiment is null;


In [ ]:
# Reading the table again from SILVER Dataset
review_df = bpd.read_gbq(f"{PROJECT_ID}.{BQ_SILVER_DATASET}.T_APP_REVIEWS_CLEANED")

In [59]:
review_df.isna().sum()

id             0
app_name       0
app_genre      0
review_text    0
sentiment      0
dtype: Int64

* With the above results we can see that the Review table is perfectly clean for further use.
* One issue is the sentiment column having a in-Appropriate value '[Negative]' which is fixed in below cell.

**Please make sure the PROJECT_NAME is replaced with your project_id before executing the below cell**

In [ ]:
%%bigquery
update `PROJECT_NAME.APP_MARKET_SILVER.T_APP_REVIEWS_CLEANED` set
sentiment='Negative' where sentiment='[Negative]';